In [1]:
import random, string
from telethon import TelegramClient, sync, events
from telethon.tl.types import Chat, User, PeerUser

In [2]:
from pyriodic import DurationJob
from pyriodic import DatetimeJob
from pyriodic import DatetimesJob
from pyriodic import Scheduler
from pyriodic import parse
from datetime import datetime, timedelta
from copy import copy
from time import sleep

In [8]:
def get_id_from_string(str):
    try:
        id = int(str)
        return id
    except:
        pass
    
def get_whitelist(file_name):
    file = open('data/{}'.format(file_name), 'r')
    content = file.read()
    whitelist_with_None = list(map(get_id_from_string, content.split('\n')))
    whitelist = [x for x in whitelist_with_None if x]
    return whitelist

def get_schedule(file_name):
    file = open('schedules/{}'.format(file_name), 'r')
    content = file.read()
    schedule_with_None = content.split('\n')
    schedule = [x for x in schedule_with_None if x]
    return schedule

def get_credentials(file_name):
    file = open('data/{}'.format(file_name), 'r')
    content = file.read()
    credentials_with_None = content.split('\n')
    credentials = [x for x in credentials_with_None if x]
    return {'api_id': int(credentials[0]), 'api_hash': credentials[1]}

def get_chats():
    chats = []
    for dialog in client.get_dialogs():
        if dialog.is_group:
            chats.append(dialog)
    return chats

def get_chat_names():
    return [chat.name for chat in get_chats()]

def get_mode_message():
    return 'В каком режиме хотите разослать спам?\n1. Одноразовый\n2. По расписанию\n\n Необходимо ввести **1** или **2**'

def get_after_mode_message(mode):
    if mode == '1':
        return 'Вы уверены, что хотите разослать это сообщение по чатам:\n{}\n...?\n\nНапишите **"да"** или **"нет"**'.format(
            '\n'.join(get_chat_names()[:5]))
    elif mode == '2':
        return '''Введите название файла с расписанием.
        
В файле должен быть список меток времени, каждая с новой строки, в формате: **МЕС.ДЕНЬ.ГОД.ЧАС:МИНУТА**
        
Например, **12.31.2019.19:30**'''

def get_error_file_schedule_message(file_name):
    return '''В файле **"{0}"** должен быть список меток времени, каждая с новой строки, в формате:
**МЕС.ДЕНЬ.ГОД.ЧАС:МИНУТА**.

Исправьте ошибки в файле **"{0}"** и попробуйте ещё раз или введите название другого файла.'''.format(file_name)
                                

def get_confirm_schedule_message(schedule):
    return '''Вы уверены, что хотите назначить рассылку по чатам:
    \n{0}\n...\n\nСогласно расписанию:\n{1}\n...\n\nНапишите **"да"** или **"нет"**'''.format('\n'.join(get_chat_names()[:5]), '\n'.join(schedule[:5]))

def get_report_message():
    if spam_mode == '1':
        return 'Сообщение разослано по чатам:\n{}\n...'.format('\n'.join(get_chat_names()[:5]))
    if spam_mode == '2':
        return 'Сообщение будет разослано по чатам:\n{0}\n...\n\nСогласно расписанию:\n{1}\n...'.format('\n'.join(get_chat_names()[:5]),'\n'.join(spam_schedule[:5]))
    

def get_autoreply_message(file_name):
    return open('data/{}'.format(file_name), 'r').read()

def validate_message(message, allowed_messages, from_id):
    if message in allowed_messages:
        return True
    else:
        allowed_str = ' или '.join(list(map(lambda message: '**"{}"**'.format(message), allowed_messages)))
        client.send_message(from_id, 'Необходимо ввести {}'.format(allowed_str), parse_mode='md')
        return False

In [4]:
# Use your own values here
api_id = get_credentials('credentials.txt')['api_id']
api_hash = get_credentials('credentials.txt')['api_hash']

client = TelegramClient('session', api_id, api_hash)
client.start()

In [5]:
s = Scheduler()

In [6]:
def dispatch(m):
    successes = 0
    errors = 0
    for chat in get_chats():
        try:
            client.send_message(chat, m)
            successes += 1
            with open("log/autoposter_log.txt", "a") as file:
                file.write(f'{datetime.now()}  {chat.name}  success\n')
        except Exception as e:
            errors += 1
            with open("log/autoposter_log.txt", "a") as file:
                file.write(f'{datetime.now()}  {chat.name}  error ({e})\n')
            for user_id in get_whitelist('whitelist.txt'):
                client.send_message(user_id, 'Не получилось отправить сообщение в группу "{}"'.format(chat.name))
        finally:
            sleep(random.randint(30,61))
    with open("log/autoposter_log.txt", "a") as file:
        file.write('Успешных рассылок: {0}  Ошибок: {1}\n\n'.format(successes, errors))
    for user_id in get_whitelist('whitelist.txt'):
                client.send_message(user_id, 'Успешных рассылок: {0}\nОшибок: {1}\n\n'.format(successes, errors))

In [7]:
spam_message = None
spam_mode = None

spam_schedule = []

@client.on(events.NewMessage())
def handler(event):
    global spam_message
    global spam_mode
    
    global spam_schedule

    if isinstance(event.message.to_id, PeerUser) and event.message.from_id != client.get_me().id:
        if event.message.from_id in get_whitelist('whitelist.txt'):
            if spam_message is None:
                spam_message = event.message
                client.send_message(event.message.from_id, get_mode_message())
            elif spam_mode is None:
                if validate_message(event.message.message.strip().lower(), ['1', '2'], event.message.from_id):
                    spam_mode = event.message.message.strip().lower()
                    client.send_message(event.message.from_id, get_after_mode_message(spam_mode), parse_mode='md')
            else:
                if spam_mode == '1':
                    if validate_message(event.message.message.strip().lower(), ['да', 'нет'], event.message.from_id):
                        if event.message.message.strip().lower() == 'да':
                            dispatch(spam_message)
                            client.send_message(event.message.from_id, get_report_message(), parse_mode='md')
                            spam_message = None
                            spam_mode = None
                        elif event.message.message.strip().lower() == 'нет':
                            client.send_message(event.message.from_id, 'Вы отменили рассылку')
                            spam_message = None
                            spam_mode = None
                elif spam_mode == '2':
                    if len(spam_schedule) == 0:
                        try:
                            schedule = get_schedule(event.message.message)
                            for elem in schedule:
                                if parse.datetime(elem) > datetime.today(): # Проверка корректности ввода
                                    spam_schedule.append(elem)
                            if len(spam_schedule) == 0:
                                client.send_message(
                                    event.message.from_id,
                                    'Файл **"{}"** либо пуст, либо в нём полностью устаревшее расписание. Исправьте его и попробуйте заново или введите название другого файла'.format(event.message.message),
                                    parse_mode='md')
                            else:
                                client.send_message(event.message.from_id, get_confirm_schedule_message(spam_schedule), parse_mode='md')
                        except FileNotFoundError:
                            client.send_message(
                                event.message.from_id,
                                'Файл **"{}"** не найден. Введите правильное название.'.format(event.message.message),
                                parse_mode='md')
                        except ValueError:
                            client.send_message(
                                event.message.from_id,
                                get_error_file_schedule_message(event.message.message),
                                parse_mode='md')
                    else:
                        if validate_message(event.message.message.strip().lower(), ['да', 'нет'], event.message.from_id):
                            if event.message.message.strip().lower() == 'да':
                                pending_message = spam_message
                                for elem in spam_schedule:
                                    job_name = 'Job_{}'.format(''.join(random.choices(string.ascii_uppercase + string.digits, k=20)))
                                    s.add_job(DatetimeJob(
                                        lambda: dispatch(pending_message),
                                        repeating=False,
                                        when=elem,
                                        name=job_name))
                                client.send_message(event.message.from_id, get_report_message(), parse_mode='md')
                                spam_message = None
                                spam_mode = None
                                spam_schedule = []
                            elif event.message.message.strip().lower() == 'нет':
                                client.send_message(event.message.from_id, 'Вы отменили рассылку')
                                spam_message = None
                                spam_mode = None
                                spam_schedule = []
        else:
            if event.message.message.strip().lower() == 'get_id':
                event.reply(str(event.message.from_id))
            else:
                client.get_dialogs()
                for user_id in get_whitelist('whitelist.txt'):
                    client.forward_messages(user_id, event.message)
                client.send_message(event.message.from_id, get_autoreply_message('autoreply.txt'))